## 12.3 PCAのアルゴリズム


## 12.4 選択と抽出の比較
### 12.4.1 特徴選択の実装

まずは特徴選択の実装をしていきます｡今回も**iris**のデータセットを利用して､SVMによる分類を行います｡

In [1]:
#写経【1】
# 必要なライブラリを読み込み
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

UsageError: Line magic function `%` not found.


In [ ]:
#写経【２】
# irisデータのロード
from sklearn.datasets import load_iris
iris = load_iris()

#データとラベルに分ける
X = iris.data
y = iris.target

In [ ]:
#写経【３】
# データに含まれている特徴量を確認
feature_names = iris.feature_names

iris.feature_names

In [ ]:
#写経【４】
# Xを確認(１０個)

X[:10]

In [ ]:
# 特徴選択による可視化
# 4つの特徴量のうち､2つを選んで2次元にする
import matplotlib.pyplot as plt

# 4つの特徴量の組み合わせを定義（合計6通り）
# 0: sepal length
# 1: sepal width
# 2: petal length
# 3: petal width
pairs = [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]

# 各組み合わせに対して散布図を表示
# グラフのサイズを指定
plt.figure(figsize=(12, 9))

# グラフを特徴量のpairごとに表示
# クラスラベルごとに色を分けてプロット
# 赤: setosa 
# 青: versicolor
# 緑: verginica
for i, (p0, p1) in enumerate(pairs):
    plt.subplot(2, 3, i + 1)
    # クラスラベルごとに色分けしてプロット
    for target, marker, color in zip(list(range(3)), ">ox", "rgb"):
        plt.scatter(X[iris.target == target, p0], X[iris.target == target, p1], marker=marker, c=color)
    plt.xlabel(feature_names[p0])
    plt.ylabel(feature_names[p1])
    plt.xticks([])
    plt.yticks([])

plt.show()

### 12.4.2 特徴抽出の実装
次は､**<font color='red'>PCA</font>**を用いて**特徴抽出**を行います｡**sklearn**の**decomposition**の中にある､**PCAクラス**を利用して行います｡

![](https://ai-std-contents.azureedge.net/image/ml12_15.png)

In [ ]:
#写経【５】
# データの標準化
#StandardScalerのライブラリを読み込む
from sklearn.preprocessing import StandardScaler
#StandardScalerのインスタンスを生成
scaler = StandardScaler()

# 与えられた行列の各特徴量について､平均と標準偏差を算出
scaler.fit(X)
# Xを標準化した行列を生成
X_std = scaler.fit_transform(X) 

In [ ]:
#写経【６】
# 主成分分析を実行
#PCAを読み込み
from sklearn.decomposition import PCA
# PCAのインスタンスを生成し、主成分を4つまで取得
pca = PCA(n_components=4)
X_pca = pca.fit_transform(X_std)

In [ ]:
#写経【７】
# 抽出した特徴量の値を出力(１０個)
X_pca[:10]

### 12.4.3 可視化による比較

寄与率の大きい順に2つの主成分を選んで（$PC_1$ と $PC_2$）可視化をしてみます｡これが､**<font color='red'>irisのデータセットを要約した特徴空間</font>**になっています｡

In [ ]:
# 特徴抽出による可視化
# PC0とPC1について散布図を表示
plt.figure(figsize=(6, 6))
for target, marker, color in zip(range(3), '>ox', 'rgb'):
    plt.scatter(X_pca[iris.target==target, 0], X_pca[iris.target==target, 1], marker=marker, color=color)
plt.xlabel('PC 0')
plt.ylabel('PC 1')
plt.xticks([])
plt.yticks([])
plt.show()

PCAの結果は以上のようになりました｡こちらが**irisのデータを要約した結果**です｡しかし､**特徴選択の場合のプロットと見比べてもそれほど大きな差異があるようには思えません**｡そこで､次節で**<font color='red'>より厳密な数値での指標をもとに効果を検証します</font>**｡

![](https://ai-std-contents.azureedge.net/image/ml12_16.png)

## 12.5 PCAの評価指標


PCAの結果を定量的に評価するために､**代表的な評価指標**について学びます｡今回取り扱うのは､以下の3つの指標です｡

1. 寄与率
2. 累積寄与率
3. 因子負荷量

### 12.5.1 寄与率

寄与率とは､**<font color='red'>ひとつの主成分がデータ全体の情報をどれだけの割合拾えているか</font>**を表す指標です｡数式的には､**<font color='red'>各主成分の持つ固有値の比率</font>**を表します｡固有値とは､線形代数に登場する $\lambda$ などの文字で表される指標です｡主成分分析においては､**この固有値が各主成分の持つ情報量を表す指標として利用できます**｡定義より､寄与率は**0~1**の値を取り､**その総和は1になります**｡**<font color='red'>寄与率が高ければ高いほど､その主成分はデータを効率よく要約する良い主成分であるといえます</font>**｡

![](https://ai-std-contents.azureedge.net/image/ml12_17.1.png)

In [ ]:
#写経【８】
# 寄与率を出力
print(pca.explained_variance_ratio_)

### 12.5.2 累積寄与率

累積寄与率とは､その名の通り**<font color='red'>寄与率の累積値</font>**であり､**<font color='red'>これまでに採用した主成分すべてで拾えた情報が合計でどの程度になるか</font>**を表す指標になります｡また､寄与率と累積寄与率は以下のような図で表現することができます｡**各主成分は互いに直行しているため､互いの拾う情報に重複がないところがポイントです**｡

![](https://ai-std-contents.azureedge.net/image/ml12_17.2.png)

![](https://ai-std-contents.azureedge.net/image/ml12_17.png)

In [ ]:
#写経【９】
# 累積寄与率を出力
print(np.cumsum(pca.explained_variance_ratio_))

さて､80:20の法則によれば､**第3､第4主成分はほとんど分析に貢献しないものとされます**｡それが本当なのか､可視化をして確認してみましょう｡

In [ ]:
# 特徴抽出による可視化
# PC3とPC4について散布図を表示
plt.figure(figsize=(6, 6))
for target, marker, color in zip(range(3), '>ox', 'rgb'):
    plt.scatter(X_pca[iris.target==target, 2], X_pca[iris.target==target, 3], marker=marker, color=color)
plt.xlabel('PC 2')
plt.ylabel('PC 3')
plt.xticks([])
plt.yticks([])
plt.show()

下に**上位の主成分との比較画像**を用意しました｡出力した結果､下位の主成分は目で見ても明らかに分類に貢献しなさそうなことがわかります｡それだけ､**上位の主成分が情報量を効率よく吸収していることがわかります**｡irisデータは4次元と低次元なデータなのでそれほど心配はいりませんが､**<font color='red'>特徴量が多いデータになるほど次元削減を行うことの利益は大きくなります</font>**｡

### 12.5.3 因子負荷量

因子負荷量（主成分負荷量）とは､**<font color='red'>主成分がもとの特徴量をどのように合成して作られたものなのかを表す指標</font>**です｡それぞれの主成分に対して､**もとの各特徴量との相関係数にあたる数値**が割り当てられており､**-1~+1**の値を取ります｡因子負荷量を参照することで､**<font color='red'>導かれた主成分の意味付けを行うことができます</font>**｡ひとつの主成分が､**どのような特徴量と相関が強いのか（軸が同じ向きや逆の向きになっているのか）**､あるいは**どのような特徴量とは相関がないのか（軸が直行しているかどうか）**に着目することで､主成分の表す意味が浮かび上がってきます｡

![](https://ai-std-contents.azureedge.net/image/ml12_20.png)

数式的には､**主成分に含まれる各特徴量の係数 $h_j$ に､その主成分の固有値 $l_i$ の平方根  $\sqrt{l_i}$ を掛け算して表現されます**｡重要なのは､ $h_j$ を利用して定義されていることから､**<font color='red'>もともとの特徴量との相関が考慮されていること</font>**が確認できることです｡

![](https://ai-std-contents.azureedge.net/image/ml12_21.png)

このようにして主成分に意味付けを行うことができれば､各サンプルを従来の特徴量の値ではなく**<font color='red'>各主成分 $PC_i$ の値（主成分スコア）を用いて評価することができます</font>**｡つまりは､**｢この学生は地頭がどの程度良いか？どれだけ理系寄りか？｣**であったり､**｢どれだけ体格ががっちりしているか？どれだけスレンダーか？｣**という評価の仕方が出来るのです｡

In [ ]:
#写経【1０】
# 主成分の係数h_jを出力
pca.components_

In [ ]:
#写経【1１】
# 固有値lのルートを取る
np.sqrt(pca.explained_variance_)

In [ ]:
#写経【1２】
# 因子負荷量を出力
pca.components_ * np.sqrt(pca.explained_variance_)[:, np.newaxis]

## 12.6 分類精度への貢献

これまでに利用してきた特徴選択とPCAによる特徴抽出を比較して､**<font color='red'>分類精度がどれだけ上がるのか</font>**を確認します｡両者において**交差検証**を行います｡

### 12.6.1 特徴選択の場合

まずは特徴選択です｡

In [ ]:
#写経【1３】
# PCA前のデータを確認
# 標準化をしたとこまでのデータを利用
X_std[:10]

In [ ]:
#写経【1４】
# 特徴量選択のために､特徴量を確認
iris.feature_names

In [ ]:
#写経【1５】
# 特徴選択(１０個)
X_std[:, 0:2][:10]

In [ ]:
#写経【1６】
# SVMで分類
#SCMのクラスを読み込む
from sklearn.svm import SVC
#モデルを評価するクラスを読み込む
from sklearn.model_selection import cross_val_score
#インスタンスを生成する
scores_1 = cross_val_score(SVC(), X_std[:, [0,2]], y, cv=5)
#平均値を求める
scores_1.mean()

### 12.6.2 特徴抽出の場合

次に､**特徴抽出**を行って実行します｡こちらも同様に2つの特徴量を利用して行います｡

In [ ]:
#写経【17】
# PCA後のデータを確認(１０個)

X_pca[:10]

In [ ]:
#写経【1８】
# 第2主成分までを選択(１０個)

X_pca[:, 0:2][:10]

In [ ]:
#写経【1９】
# SVMで分類
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
scores_2 = cross_val_score(SVC(), X_pca[:, [0,2]], y, cv=5)
scores_2.mean()

両者を比較すると､特徴抽出の方が2%ほど性能が高いことがわかります｡

In [ ]:
#写経【２０】
# 両者の精度を比較
#特徴選択
print('特徴選択: {}'.format(scores_1.mean()))
#特徴抽出
print('特徴抽出: {}'.format(scores_2.mean()))